In [13]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

print(tf.__version__)

imdb = tf.keras.datasets.imdb

# train data = 25,000, test_data = 25,000
# num_words -> data size 유지. (상위 자주 등장하는 10,000개의 단어)
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
print("train : ", train_data.shape, train_labels.shape)
print("test : ", test_data.shape, test_labels.shape)


2.1.0
train :  (25000,) (25000,)
test :  (25000,) (25000,)


In [14]:
# data 는 어휘사전의 특정 단어를 나타내는 정수
# labels은 0, 1로 구성. 0(부정) , 1(긍정)
print(train_data[0])
print(train_labels[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
1


In [15]:
# 단어와 정수 index 매치
word_index = imdb.get_word_index()

# 3개의 index 추가
# 정수로 표현된 data_set 을 문자열로 변환.
# word_index -> "문자열" : index
word_index = {k: (v + 3) for k, v in word_index.items()}
word_index["<PAD>"] = 0  # 공백
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

# key 와 value 값 바꾸어 줌
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])


# index 와 문자열 mapping
def decode_review(text):
    return " ".join([reverse_word_index.get(i, "?") for i in text])


print(decode_review(train_data[0]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

In [16]:
# data processing 과정. tensor 로 변환 해주어야함
# sol 1. one-hot encoding. data size -> num_word * train_data size
# sol 2. padding. data size -> max_length * train_data
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
train_data = keras.preprocessing.sequence.pad_sequences(
    train_data, value=0, padding="post", maxlen=256
)

test_data = keras.preprocessing.sequence.pad_sequences(
    test_data, value=0, padding="post", maxlen=256
)

print(len(train_data[0]), len(test_data[0]))
print(train_data[0])

256 256
[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  10

In [17]:
# layer 구성하기.
# 모델에서 얼마나 많은 층을 사용할 것인가?
# 각 층에서 얼마나 많은 은닉 유닛(hidden unit)을 사용할 것인가?
vocab_size = 10000
# model = tf.keras.Sequential(
#     [
#         tf.keras.layers.Embedding(vocab_size,16), # voca 사이즈 = word_num, 16차원
#         tf.keras.layers.GlobalAveragePooling1D(),
#         tf.keras.layers.Dense(16,activation="relu"),
#         tf.keras.layers.Dense(1,activation="sigmoid") # 0과 1
#     ]
# )
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16, input_shape=(None,)))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [18]:
# 손실 함수, optimizer
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [19]:
# validation set 만들기 -> train_set 에서 학습 및 검증
# test data는 마지막에 한번 만 검증 용으로 사용
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

# 모델 훈련
# batch 사이즈는 메모리에 맞게 적절히 설정
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=40,
    batch_size=512,
    validation_data=(x_val, y_val),
    verbose=1,
)

Train on 15000 samples, validate on 10000 samples
Epoch 1/40
15000/15000 [==============================] - 4s 264us/sample - loss: 0.6923 - accuracy: 0.5449 - val_loss: 0.6906 - val_accuracy: 0.6404
Epoch 2/40
15000/15000 [==============================] - 2s 133us/sample - loss: 0.6877 - accuracy: 0.6948 - val_loss: 0.6842 - val_accuracy: 0.7372
Epoch 3/40
15000/15000 [==============================] - 2s 128us/sample - loss: 0.6775 - accuracy: 0.7394 - val_loss: 0.6709 - val_accuracy: 0.7208
Epoch 4/40
15000/15000 [==============================] - 2s 138us/sample - loss: 0.6592 - accuracy: 0.7578 - val_loss: 0.6502 - val_accuracy: 0.7125
Epoch 5/40
15000/15000 [==============================] - 2s 120us/sample - loss: 0.6317 - accuracy: 0.7701 - val_loss: 0.6187 - val_accuracy: 0.7593
Epoch 6/40
15000/15000 [==============================] - 2s 124us/sample - loss: 0.5917 - accuracy: 0.8011 - val_loss: 0.5788 - val_accuracy: 0.8013
Epoch 7/40
15000/15000 [==========================

In [20]:
results = model.evaluate(test_data,  test_labels, verbose=2)

print(results)


25000/25000 - 2s - loss: 2.2907 - accuracy: 0.4966
[2.2907491260528565, 0.49664]
